In [ ]:
string clientId = "";
string clientSecret = "";
string tenantId = "";
string resourceGroup = "";
string subscriptionId = "";

In [ ]:
#r "nuget: Azure.Identity, *-*"
#r "nuget: Azure.ResourceManager.Compute, *-*"
#r "nuget: Azure.ResourceManager.Network, *-*"
#r "nuget: Azure.ResourceManager.Resources, *-*"

In [ ]:
using Azure.Identity;
using Azure.ResourceManager;

var credentials = new ClientSecretCredential(tenantId, clientId, clientSecret);
var armClient = new ArmClient(credentials, subscriptionId);

In [ ]:
var subscription = armClient.GetDefaultSubscription();
var resource = subscription.GetResourceGroup(resourceGroup).Value;

In [ ]:
using Azure.ResourceManager.Compute;

var vmCollection = resource.GetVirtualMachines();
var vms = vmCollection.GetAll();

foreach (var vm in vms)
{
    Console.WriteLine($"{vm.Id} - {vm.Data.Name}");

    foreach(var n in vm.Data.NetworkProfile.NetworkInterfaces)
        Console.WriteLine($"\t{n.Id} - {n.Primary}");
}

In [ ]:
using Azure.ResourceManager.Compute;

string resourceGroup = "";
string virtualMachineName = "";

var resource = subscription.GetResourceGroup(resourceGroup).Value;
var vm = resource.GetVirtualMachine(virtualMachineName).Value;
vm

In [ ]:
using Azure.Core;

string resourceId = "";
var id = new ResourceIdentifier(resourceId);

Console.WriteLine($"Name: {id.Name}");
Console.WriteLine($"Subscription: {id.SubscriptionId}");
Console.WriteLine($"ResourceGroup: {id.ResourceGroupName}");

In [ ]:
// https://github.com/Azure/azure-sdk-for-net/blob/main/sdk/compute/Azure.ResourceManager.Compute/samples/Sample2_ManagingVirtualMachines.md

var op = await vm.PowerOnAsync(Azure.WaitUntil.Completed); //another option is to use Azure.WaitUntil.Started
op.HasCompleted

In [ ]:
var op = await vm.DeallocateAsync(Azure.WaitUntil.Completed);
op.HasCompleted

In [ ]:
var disks = resource.GetManagedDisks();
foreach(var disk in disks.GetAll())
    Console.WriteLine(disk.Data.Name);

In [ ]:
// https://github.com/Azure/azure-sdk-for-net/blob/Azure.ResourceManager.Compute_1.1.0/sdk/compute/Azure.ResourceManager.Compute/samples/Generated/Samples/Sample_VirtualMachineResource.cs

// https://github.com/Azure/azure-sdk-for-net/blob/Azure.ResourceManager.Compute_1.1.0/sdk/compute/Azure.ResourceManager.Compute/samples/Sample1_ManagingDisks.md
// https://github.com/Azure/azure-sdk-for-net/blob/Azure.ResourceManager.Compute_1.1.0/sdk/compute/Azure.ResourceManager.Compute/samples/Generated/Samples/Sample_SnapshotResource.cs

using Azure;
using Azure.ResourceManager.Compute.Models;

var osDisk = vm.Data.StorageProfile.OSDisk;
var disk = osDisk.ManagedDisk;

var snapshotName = "";
var snapshots = resource.GetSnapshots();
var snapshotData = new SnapshotData(vm.Data.Location)
{
    CreationData = new DiskCreationData(DiskCreateOption.Copy)
    {
        SourceResourceId = disk.Id
    }
};

var op = snapshots.CreateOrUpdate(WaitUntil.Completed, snapshotName, snapshotData);
var snapshot = op.Value;
snapshot